In [1]:
# import os
# import shutil
# import logging
# from git import Repo
# import pandas as pd
# import numpy as np
# from datetime import datetime
# from pytz import timezone
# from kaggle_secrets import UserSecretsClient
# import psutil
# print(f"RAM Usage: {psutil.virtual_memory().percent}%")


# # Configure Logging
# logging.basicConfig(
#     filename="pipeline.log",
#     level=logging.INFO,
#     format="%(asctime)s - %(levelname)s - %(message)s"
# )

# def load_data(file_path):
#     """Load dataset from JSON file."""
#     try:
#         df = pd.read_json(file_path)
#         logging.info(f"Dataset loaded successfully from {file_path}")
#         return df
#     except Exception as e:
#         logging.error(f"Failed to load dataset: {e}")
#         raise


# def pre_eda_validation(df, batch_size=1000):
#     """Perform pre-EDA validation in batches."""
#     try:
#         total_batches = (len(df) // batch_size) + 1
#         report_list = []

#         logging.info(f"Processing DataFrame in {total_batches} batches...")

#         for i in range(total_batches):
#             batch = df.iloc[i * batch_size:(i + 1) * batch_size]
#             batch_report = {
#                 'missing_values': batch.isnull().sum(),
#                 'duplicates': batch.duplicated().sum(),
#                 'data_types': batch.dtypes.astype(str),
#                 'cardinality': batch.nunique()
#             }
#             report_list.append(pd.DataFrame(batch_report))

#         final_report = pd.concat(report_list).reset_index(drop=True)

#         if not final_report.empty:
#             timestamp = datetime.now(ist).strftime("%Y-%m-%d_%H_%M_%S")
#             filename = f"PEDA_{timestamp}_{len(final_report)}_records.json"
#             final_report.to_json(filename, orient="index", indent=4)
#             logging.info(f"DataFrame saved: {filename}")
#             return filename
#         else:
#             logging.info("No data to save since empty DataFrame returned.")
#             return None

#     except Exception as e:
#         logging.error(f"Pre-EDA validation failed: {e}")
#         raise



# def detect_outliers_iqr(df, column):
#     """Detect and handle outliers using IQR method."""
#     try:
#         Q1 = df[column].quantile(0.25)
#         Q3 = df[column].quantile(0.75)
#         IQR = Q3 - Q1
#         lower_bound = max(Q1 - 1.5 * IQR, df[column].min())
#         upper_bound = min(Q3 + 1.5 * IQR, df[column].max())
#         df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
#         df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
#         logging.info(f"Outliers handled in column: {column}")
#         return df
#     except Exception as e:
#         logging.error(f"Failed to detect outliers in {column}: {e}")
#         raise

# # def GitHubPush(file, filename, kaggle_repo_url, destination_path):
# #     try:
# #         if not os.path.exists(destination_path):
# #             os.makedirs(destination_path)
# #             logging.info(f"Created directory: {destination_path}")

# #         record_count = len(file)
# #         timestamp = datetime.now(ist).strftime("%Y-%m-%d_%H_%M_%S")
# #         filename = f"{filename}_{timestamp}_{record_count}_records.json"
# #         file.to_json(filename, orient="records", indent=4)
# #         shutil.copyfile(filename, f"{destination_path}/{filename}")

# #         repo = Repo(kaggle_repo_url)
# #         repo.index.add([f"{destination_path}/{filename}"])
# #         repo.index.commit(f"{timestamp} - Added {filename}")
# #         repo.remote(name="origin").push()
# #         logging.info(f"Pushed file: {filename} to GitHub")

# #     except Exception as e:
# #         logging.error(f"GitHub push failed: {e}")
# #         raise

# def main(df, destination_path, kaggle_repo_url):
#     try:
#         report = pre_eda_validation(df)
#         # GitHubPush(report, "PEDA", kaggle_repo_url, destination_path)

#         numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
#         for col in numeric_cols:
#             df = detect_outliers_iqr(df, col)

#         duplicates = df[df.duplicated('videoTitle', keep=False)][['videoTitle', 'channelName']]
#         logging.info(f"Duplicate Titles Found: {len(duplicates)}")
#         print(duplicates)
        
#     except Exception as e:
#         logging.error(f"Pipeline execution failed: {e}")

# if __name__ == "__main__":
#     try:
#         user_secrets = UserSecretsClient()
#         name = user_secrets.get_secret("EDARepoOwner")
#         email = user_secrets.get_secret("EDARepoOwnerMail")
#         repo_url = user_secrets.get_secret("EDARepoURL")
        
#         kaggle_repo_url = '/kaggle/working/DevOps-YouTube-Trends'
#         dataset_path = '/kaggle/working/DevOps-YouTube-Trends/FeatureEngineering/Daily'
#         destination_path = '/kaggle/working/DevOps-YouTube-Trends/ExploratoryDataAnalysis/Daily'

#         ist = timezone('Asia/Kolkata')
#         year = datetime.now(ist).strftime("%Y")
#         today = datetime.now(ist).strftime("%Y-%m-%d")
#         destination_path = f"{destination_path}/{year}/{today}"

#         if not os.path.exists(kaggle_repo_url):
#             Repo.clone_from(repo_url, kaggle_repo_url)
#             logging.info("Repository cloned successfully.")
#         else:
#             repo = Repo(kaggle_repo_url)
#             repo.remote(name="origin").pull()
#             logging.info("Repository updated successfully.")

#         datasets = os.listdir(dataset_path)
#         Target_Dataset = max([d for d in datasets if d.startswith("FE_") and d.endswith('records.json')])
#         file_path = os.path.join(dataset_path, Target_Dataset)
#         df = load_data(file_path)
#         main(df, destination_path, kaggle_repo_url)

#     except Exception as e:
#         logging.critical(f"Main pipeline execution failed: {e}")


In [2]:
import os
import shutil
from git import Repo
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
import json
from kaggle_secrets import UserSecretsClient

In [3]:
def FeatureEngineering_File_Extraction(repo_url, kaggle_repo_url, FeatureEngineering_path):
    # Check if the repository already exists locally
    if os.path.exists(kaggle_repo_url):
        print("Repository already exists locally.")
        repo = Repo(kaggle_repo_url)  # Access the existing repository
        repo.config_writer().set_value("user", "name", name).release()
        repo.config_writer().set_value("user", "email", email).release()
        origin = repo.remote(name='origin')  # Get the remote repository reference
        origin.pull()  # Pull the latest updates from the remote repository
        print("Successfully pulled the latest changes.")
    else:
        # Clone the repository if it does not exist
        repo = Repo.clone_from(repo_url, kaggle_repo_url)
        repo.config_writer().set_value("user", "name", name).release()
        repo.config_writer().set_value("user", "email", email).release()
        print("Successfully cloned the repository.")

    # List all files in the specified directory and filter for the relevant JSON file
    output_files = os.listdir(FeatureEngineering_path)
    FeatureEngineering_File = max(
        [file for file in output_files if file.startswith("FE_") and file.endswith('records.json')]
    )

    # Read the identified JSON file into a pandas DataFrame
    FeatureEngineering_File = pd.read_json(os.path.join(FeatureEngineering_path, FeatureEngineering_File))

    return FeatureEngineering_File

In [4]:
def PushToGithub(filename,destination_path):
    try:
        if os.path.exists(kaggle_repo_url):
            print("Already cloned and the repo file exists")
            repo = Repo(kaggle_repo_url)
            repo.config_writer().set_value("user", "name", name).release()
            repo.config_writer().set_value("user", "email", email).release()
            origin = repo.remote(name='origin')
            origin.pull()
            print("Successfully pulled the git repo before push")
        else:
            repo = Repo.clone_from(repo_url, kaggle_repo_url)
            repo.config_writer().set_value("user", "name", name).release()
            repo.config_writer().set_value("user", "email", email).release()
            print("Successfully cloned the git repo")
        
        if os.path.exists(destination_path):
            shutil.copyfile(f'/kaggle/working/{filename}', f'{destination_path}/{filename}')
        else:
            os.makedirs(destination_path)
            shutil.copyfile(f'/kaggle/working/{filename}', f'{destination_path}/{filename}')
        
        repo = Repo(kaggle_repo_url)
        repo.index.add([f"{destination_path}/{filename}"])
        timestamp = datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
        repo.index.commit(f"{timestamp} Added files from Kaggle notebook, {filename}")
        origin = repo.remote(name="origin")
        push_result = origin.push()
        
        if push_result:
            print("Output files successfully pushed to GitHub!")
        else:
            print("Output files pushed to GitHub failed:(")
        return True
    
    except Exception as e:
        print(f"An error occurred at git automation code: {e}")
        return False

In [5]:
# def pre_eda_validation(dataFrame):
#     timestamp = datetime.now(ist).strftime("%Y-%m-%d_%H_%M_%S")
#     record_count = len(dataFrame)

#     report_df = pd.DataFrame({
#         "missing_values": dataFrame.isnull().sum(),
#         "duplicates": [dataFrame.duplicated().sum()] * len(dataFrame.columns),
#         "data_types": dataFrame.dtypes.astype(str),
#         "cardinality": dataFrame.nunique()
#     }).reset_index().rename(columns={"index": "columns"})

#     filename = f"PEDA_{timestamp}_{record_count}_records.json"

#     if not report_df.empty:
#         report_df.to_json(filename, orient="records", indent=4)
#         print(f"DataFrame validation report saved as {filename}")
#     else:
#         print("No data to save since empty DataFrame returned.")

#     destination_path = '/kaggle/working/DevOps-YouTube-Trends/ExploratoryDataAnalysis/PEDA/Daily'  
#     PushToGithub(filename,destination_path)
        
#     return True

In [6]:
def pre_eda_validation(dataFrame):
    timestamp = datetime.now(ist).strftime("%Y-%m-%d_%H_%M_%S")
    record_count = len(dataFrame)

    report_df = pd.DataFrame({
        "missing_values": dataFrame.isnull().sum(),
        "duplicates": [dataFrame.duplicated().sum()] * len(dataFrame.columns),
        "data_types": dataFrame.dtypes.astype(str),
        "cardinality": dataFrame.nunique()
    }).reset_index().rename(columns={"index": "columns"})

    # Extract inconsistent records for group1
    inconsistent_group1 = dataFrame[dataFrame.duplicated(subset=["channelId"], keep=False)][
        ["channelId", "channelName", "channelCustomUrl", "channelGrowthScoreRank"]]

    # Extract inconsistent records for group2
    inconsistent_group2 = dataFrame[dataFrame.duplicated(subset=["videoId"], keep=False)][
        ["videoId", "videoTitle", "videoEngagementScoreRank"]]

    report = {
        "Pre_EDA": report_df.to_dict(orient="records"),
        "inconsistent_records_group1": inconsistent_group1.to_dict(orient="records"),
        "inconsistent_records_group2": inconsistent_group2.to_dict(orient="records")
    }

    filename = f"PEDA_{timestamp}_{record_count}_records.json"

    if report["Pre_EDA"]:
        with open(filename, "w") as json_file:
            json.dump(report, json_file, indent=4)
        print(f"DataFrame validation report saved as {filename}")
    else:
        print("No data to save since empty DataFrame returned.")

    destination_path = '/kaggle/working/DevOps-YouTube-Trends/ExploratoryDataAnalysis/PEDA/Daily'
    PushToGithub(filename, destination_path)
        
    return True


In [7]:
def main(repo_url, kaggle_repo_url, FeatureEngineering_path, ExploratoryDataAnalysis_path):
    FeatureEngineering_File = FeatureEngineering_File_Extraction(repo_url, kaggle_repo_url, FeatureEngineering_path)
    pre_eda_validation(FeatureEngineering_File)
    return True

In [8]:
if __name__ == "__main__":    
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("EDARepoOwner")
    secret_value_1 = user_secrets.get_secret("EDARepoOwnerMail")
    secret_value_2 = user_secrets.get_secret("EDARepoURL")
    
    name = secret_value_0
    email = secret_value_1
    repo_url = secret_value_2
    
    kaggle_repo_url = '/kaggle/working/DevOps-YouTube-Trends'
    FeatureEngineering_path = '/kaggle/working/DevOps-YouTube-Trends/FeatureEngineering/Daily'
    ExploratoryDataAnalysis_path = '/kaggle/working/DevOps-YouTube-Trends/ExploratoryDataAnalysis'

    ist = timezone("Asia/Kolkata")
    
    main(repo_url, kaggle_repo_url, FeatureEngineering_path, ExploratoryDataAnalysis_path)

Successfully cloned the repository.
DataFrame validation report saved as PEDA_2025-03-05_19_08_00_392_records.json
Already cloned and the repo file exists
Successfully pulled the git repo before push
Output files successfully pushed to GitHub!
